# 背景
时间：2023.1.30 <br/>
对通用数据集，做BERT模型和Longformer模型的训练，探索BERT模型和Longformer模型在短文本领域的效果，是否符合BERT模型在短文本领域效果较好，Longformer模型在长文本领域效果较好

# 通用模块

## 参数设置

In [ ]:
model_checkpoint = "bert-base-chinese"
# model_checkpoint = "schen/longformer-chinese-base-4096"
batch_size = 2 # 每一批次的数量
num_labels = 2 # 多少分类，这里是二分类问题，积极和消极
output_dir = "/home/chenli/pre_model/BERT_and_Longformer/BERT" # 模型保存路径
learning_rate = 1e-5 # 学习率
# weight_decay=0.01 # 学习率衰减，设置0.01即可。如果weight_decay设置太小，几乎就不起作用了。
num_train_epochs = 10 # 训练轮次，差不多设置为5左右。轮数不要设置太大。轮数设置的太大，Loss是下降了，但是微调的时候效果不是很好，有可能训练过头了

## 加载数据集

In [ ]:
from datasets import load_dataset
from datasets import load_from_disk
# 加载一个评估标准，默认的评估标准
from datasets import load_metric

In [ ]:
# 加载数据集
dataset = load_from_disk('./data/ChnSentiCorp')
metric = load_metric("glue","mrpc")

In [ ]:
dataset

## 数据预处理

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
# 分词
def preprocess_function(data):
    return tokenizer(data['text'],truncation=True)

In [ ]:
encoded_dataset = dataset.map(function=preprocess_function,
                     batched=True,
                     remove_columns=['text'])

In [ ]:
encoded_dataset

## 微调预训练模型

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

In [ ]:
metric_name = "accuracy"

args = TrainingArguments(
    output_dir = output_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = learning_rate,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = 1,
    num_train_epochs = num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
import numpy as np
def compute_metrics(eval_preds):
    metric = load_metric('glue','mrpc')
    logits,labels = eval_preds # 预测值和真实值
    predictions = np.argmax(logits,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# BERT模型训练

In [ ]:
trainer.train()